In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from data import merged_data, get_education_indicators

In [2]:
indicators = get_education_indicators()
indicators.head()

indicator_code
SE.ADT.LITR.FE.ZS                                                       literacy_rate_female
SE.ADT.LITR.MA.ZS                                                         literacy_rate_male
Literacy rate, adult total (% of people ages 15 and above)               literacy_rate_total
SE.COM.DURS                                                    compulsory_education_duration
SE.PRM.CMPT.FE.ZS                                             completion_rate_primary_female
Name: indicator, dtype: object

In [7]:
df = merged_data()
df.head()

completed_education_rate_primary_female  \
country_code year                                              
ABW          2000.0                                 89.06955   
             2001.0                                 89.06955   
             2002.0                                 89.06955   
             2003.0                                 89.06955   
             2004.0                                 89.06955   

                     completed_education_rate_primary_male  \
country_code year                                            
ABW          2000.0                               91.75029   
             2001.0                               91.75029   
             2002.0                               91.75029   
             2003.0                               91.75029   
             2004.0                               91.75029   

                     completed_education_rate_primary_total  \
country_code year                                             
ABW          2000.0                                90.32299   
             2001.0                                90.32299   
             2002.0                                90.32299   
             2003.0                                90.32299   
             2004.0                                90.32299   

                     completion_rate_lower_secondary_female  \
country_code year                                             
ABW          2000.0                               52.930729   
             2001.0                               52.930729   
             2002.0                               52.930729   
             2003.0                               52.930729   
             2004.0                               52.930729   

                     completion_rate_lower_secondary_male  \
country_code year                                           
ABW          2000.0                             59.478802   
             2001.0                             59.478802   
             2002.0                             59.478802   
             2003.0                             59.478802   
             2004.0                             59.478802   

                     completion_rate_lower_secondary_total  \
country_code year                                            
ABW          2000.0                              55.992432   
             2001.0                              55.992432   
             2002.0                              55.992432   
             2003.0                              55.992432   
             2004.0                              55.992432   

                     completion_rate_post_secondary_female  \
country_code year                                            
ABW          2000.0                                    NaN   
             2001.0                                    NaN   
             2002.0                                    NaN   
             2003.0                                    NaN   
             2004.0                                    NaN   

                     completion_rate_post_secondary_male  \
country_code year                                          
ABW          2000.0                                  NaN   
             2001.0                                  NaN   
             2002.0                                  NaN   
             2003.0                                  NaN   
             2004.0                                  NaN   

                     completion_rate_post_secondary_total  \
country_code year                                           
ABW          2000.0                                   NaN   
             2001.0                                   NaN   
             2002.0                                   NaN   
             2003.0                                   NaN   
             2004.0                                   NaN   

                     completion_rate_primary_female  ...  \
country_code year                             

In [12]:
def create_levels(name):
    parts = name.split('_')
    
    school_level = ['primary', 'secondary', 'tertiary']
    gender = ['female', 'male', 'total']
    level1 = np.nan
    level2 = np.nan

    if len(parts) > 2:
        # create level 1
        if parts[-1] in school_level:
            level1 =  parts[-1]
        elif parts[-2] in school_level:
            level1 = parts[-2]
        
        # create level 2
        if parts[-1] in gender:
            level2 = parts[-1]
        
        # create level 0
        for var in school_level + gender:
            if var in name:
                name = name.replace('_' + var, '')
        
        return (name, level1, level2)

[create_levels(name) for name in df.columns]

[('completed_education_rate', 'primary', 'female'),
 ('completed_education_rate', 'primary', 'male'),
 ('completed_education_rate', 'primary', 'total'),
 ('completion_rate_lower', 'secondary', 'female'),
 ('completion_rate_lower', 'secondary', 'male'),
 ('completion_rate_lower', 'secondary', 'total'),
 ('completion_rate_post', 'secondary', 'female'),
 ('completion_rate_post', 'secondary', 'male'),
 ('completion_rate_post', 'secondary', 'total'),
 ('completion_rate', 'primary', 'female'),
 ('completion_rate', 'primary', 'male'),
 ('completion_rate', 'primary', 'total'),
 ('completion_rate', 'tertiary', 'female'),
 ('completion_rate', 'tertiary', 'male'),
 ('completion_rate', 'tertiary', 'total'),
 ('completion_rate_upper', 'secondary', 'female'),
 ('completion_rate_upper', 'secondary', 'male'),
 ('completion_rate_upper', 'secondary', 'total'),
 ('compulsory_education_duration', nan, nan),
 None,
 ('education_duration', 'primary', nan),
 ('education_expenditure_gdp_rate', nan, 'total'),
